**Грамматический анализатор для калькулятора**

Евгений Борисов <esborisov@sevsu.ru>

библиотека [Python Lex-Yacc](https://ply.readthedocs.io/en/latest/ply.html)

Виталий Дятлов  
Разбор кода и построение синтаксических деревьев с PLY.   
https://habr.com/ru/post/191252/


David Beazley    
A simple calculator with variables.   
https://pythonrepo.com/repo/dabeaz-ply-python-text-processing

In [1]:
# !pip install ply

In [2]:
__file__ = '01-ply-calc.ipynb'

In [9]:
# Build the lexer
import ply.lex as lex

tokens = (
    'INTEGER',
    'PLUS','MINUS','TIMES','DIVIDE',
    'LPAREN','RPAREN',
    )

# Tokens
t_PLUS    = r'\+'
t_MINUS   = r'-'
t_TIMES   = r'\*'
t_DIVIDE  = r'/'
t_LPAREN  = r'\('
t_RPAREN  = r'\)'

# Ignored characters
t_ignore = " \t"

def t_INTEGER(t):
    r'\d+'
    t.value = int(t.value)
    return t

def t_error(t):
    print(f"Illegal character {t.value[0]!r}")
    t.lexer.skip(1)


lexer = lex.lex()

In [5]:
import ply.yacc as yacc

# Precedence rules for the arithmetic operators
precedence = (
    ('left','PLUS','MINUS'),
    ('left','TIMES','DIVIDE'),
    ('right','UMINUS'),
    )

def p_statement_expr(p):
    'statement : expression'
    print(p[1])

def p_expression_binop(p):
    '''expression : expression PLUS expression
                  | expression MINUS expression
                  | expression TIMES expression
                  | expression DIVIDE expression'''
    if   p[2] == '+': p[0] = p[1] + p[3]
    elif p[2] == '-': p[0] = p[1] - p[3]
    elif p[2] == '*': p[0] = p[1] * p[3]
    elif p[2] == '/': p[0] = p[1] / p[3]

def p_expression_uminus(p):
    'expression : MINUS expression %prec UMINUS'
    p[0] = -p[2]

def p_expression_group(p):
    'expression : LPAREN expression RPAREN'
    p[0] = p[2]

def p_expression_number(p):
    'expression : INTEGER'
    p[0] = p[1]

def p_error(p):
    print(f"Syntax error at {p.value!r}")

parser = yacc.yacc(write_tables=False)

Generating LALR tables


In [6]:
while True:
    try:
        s = input('calc > ')
        parser.parse(s)
    except Exception as e:
        print('error:',e)
        break

calc > 3+4
7
calc > (3+7)/4
2.5
calc > 
error: 'NoneType' object has no attribute 'value'
